<a href="https://colab.research.google.com/github/MahdiEsrafili/pert/blob/master/ezafe_blstm_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install stanza

     |████████████████████████████████| 432 kB 8.9 MB/s 
     |████████████████████████████████| 175 kB 56.5 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=ef8cf517999970f4c7230cf77061f42e2e6a55efd7e444d611c587681fdfd0aa
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


In [ ]:
import stanza
stanza.download('fa')

2022-04-16 07:55:09 INFO: Downloading default packages for language: fa (Persian)...
2022-04-16 07:55:10 INFO: File exists: /root/stanza_resources/fa/default.zip.
2022-04-16 07:55:12 INFO: Finished downloading models and saved to /root/stanza_resources.


In [ ]:
nlp_pos = stanza.Pipeline(lang='fa', processors='tokenize,mwt,pos')

2022-04-16 07:55:12 INFO: Loading these models for language: fa (Persian):
| Processor | Package |
-----------------------
| tokenize  | perdt   |
| mwt       | perdt   |
| pos       | perdt   |

2022-04-16 07:55:12 INFO: Use device: gpu
2022-04-16 07:55:12 INFO: Loading: tokenize
2022-04-16 07:55:21 INFO: Loading: mwt
2022-04-16 07:55:21 INFO: Loading: pos
2022-04-16 07:55:21 INFO: Done loading processors!


In [2]:
!git clone https://github.com/facebookresearch/fastText.git
% cd fastText
!  pip install .
%cd ..

Cloning into 'fastText'...
remote: Enumerating objects: 3930, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 3930 (delta 28), reused 42 (delta 11), pack-reused 3854
Receiving objects: 100% (3930/3930), 8.33 MiB | 38.27 MiB/s, done.
Resolving deltas: 100% (2445/2445), done.
/content/fastText
Processing /content/fastText
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Using cached pybind11-2.9.2-py2.py3-none-any.whl (213 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3139533 sha256=98cc7ca5d1af18910d7b21d5387950286ea748c6c5a497c0ee1f8

In [ ]:
% cd fastText
!./download_model.py fa
% cd ..

/content/fastText
 (100.00%) [==================================================>]
/content


In [3]:
!mkdir data

In [4]:
# run this or below cell
!cp drive/MyDrive/دیتاست\ بیجن\ خان/test_clean.tsv data/test_clean.tsv

In [5]:
!unzip drive/MyDrive/دیتاست\ بیجن\ خان/test_data.zip -d data
!unzip drive/MyDrive/دیتاست\ بیجن\ خان/train_data.zip -d data

Archive:  drive/MyDrive/دیتاست بیجن خان/test_data.zip
  inflating: data/test_data.txt      
Archive:  drive/MyDrive/دیتاست بیجن خان/train_data.zip
  inflating: data/train_data.txt     


In [ ]:
nlp_pos = stanza.Pipeline(lang='fa', processors='tokenize,mwt,pos')

2022-04-16 07:55:21 INFO: Loading these models for language: fa (Persian):
| Processor | Package |
-----------------------
| tokenize  | perdt   |
| mwt       | perdt   |
| pos       | perdt   |

2022-04-16 07:55:21 INFO: Use device: gpu
2022-04-16 07:55:21 INFO: Loading: tokenize
2022-04-16 07:55:21 INFO: Loading: mwt
2022-04-16 07:55:21 INFO: Loading: pos
2022-04-16 07:55:21 INFO: Done loading processors!


In [ ]:

def pos_tager(text):
  doc = nlp_pos(text)
  pos = [word.xpos for sent in doc.sentences for word in sent.words if word.text not in ['ش','شان','م', 'مان', 'ند','ست','یت', 'تان', 'ت','اش']]
  # pos_ = [(word.text , word.upos, word.xpos) for sent in doc.sentences for word in sent.words ]
  # print(pos_)
  return pos

In [ ]:
from tqdm import tqdm

In [ ]:
def check_consist(pos_tag, kasre_tag, sent):
  l1 = len(pos_tag)
  l2 = len(kasre_tag)
  l3 = len(sent.split())
  if l1!=l2:
    return False
  if l1!=l3:
    return False
  if l3!=l2:
    return False
  return True

In [ ]:
with open('data/test_data.txt', encoding='utf-8') as f:
    data = f.readlines()
sents = list()
kasre_tags = list()
temp_sent = ''
temp_tags = list()
pos_tags = list()
for i, line in tqdm(enumerate(data)):
    # if i>10000: break
    try:
        word, ez = line.split()
        word += ' '
        temp_sent += word
        temp_tags.append(ez)
        if word in ['. ', '# ']:
            pos_tag = pos_tager(temp_sent)
            ok = check_consist(pos_tag, temp_tags, temp_sent)
            if ok:
              sents.append(temp_sent) 
              kasre_tags.append(temp_tags)
              pos_tags.append(pos_tag)
            temp_sent = ''
            temp_tags = list()
    except:
        pos_tag = pos_tager(temp_sent)
        ok = check_consist(pos_tag, temp_tags, temp_sent)
        if ok:
          sents.append(temp_sent) 
          kasre_tags.append(temp_tags)
          pos_tags.append(pos_tag)
        temp_sent = ''
        temp_tags = list()   

970it [00:01, 1102.52it/s]/usr/local/lib/python3.7/dist-packages/stanza/models/common/beam.py:86: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  prevK = bestScoresId // numWords
1525970it [19:16, 1319.64it/s]


In [ ]:
len(sents), len(kasre_tags), len(pos_tags)

(56132, 56132, 56132)

In [ ]:
raws = list()

for i in range(len(pos_tags)):
  for j in range(len(pos_tags[i])):
    raw = f'{sents[i].split()[j]}\t{pos_tags[i][j]}\t{kasre_tags[i][j]}'
    raws.append(raw)
  raws.append('\n')

In [ ]:
raws_text = '\n'.join(raws)
with open('data/test_clean.tsv', 'w') as f:
  f.write(raws_text)

In [ ]:
!cp data/test_clean.tsv drive/MyDrive/دیتاست\ بیجن\ خان/

In [6]:
# run this or bellow cell
!cp drive/MyDrive/fastText/cc.fa.300.vec fastText/

In [ ]:
from fasttext import load_model

# original BIN model loading
f = load_model('fastText/cc.fa.300.bin')
lines=[]

# get all words from model
words = f.get_words()

with open('fastText/cc.fa.300.vec','w') as file_out:
    
    # the first line must contain number of total words and vector dimension
    file_out.write(str(len(words)) + " " + str(f.get_dimension()) + "\n")

    # line by line, you append vectors to VEC file
    for w in words:
        v = f.get_word_vector(w)
        vstr = ""
        for vi in v:
            vstr += " " + str(vi)
        try:
            file_out.write(w + vstr+'\n')
        except:
            pass

In [ ]:
!mkdir drive/MyDrive/fastText
!cp fastText/cc.fa.300.vec drive/MyDrive/fastText/

In [8]:
!mkdir checkpoints

In [1]:
RUN = 20

In [2]:
class Config:
    def __init__(self):
        # directories
        self.train_data_dir = 'data/train_data.txt'
        self.model_dir = 'models'
        self.we_model_dir = 'fastText/cc.fa.300.vec'
        self.we_pickled_model_dir = 'fastText/cc.fa.300.pickle'

        # general
        self.data_split = .1
        self.num_epochs = 25
        self.batch_size = 16
        self.shuffle_buffer = 320000
        self.num_tags = 6
        self.num_pos_tags = 33
        self.word_max_len = 30
        self.learning_rate = 1e-3
        self.max_len = 100

        # embeddings
        self.num_words = 100000
        self.word_embed_dim = 300
        self.num_chars = 256  # number of most frequent characters to be kept
        self.char_embed_dim = 32
        self.pos_embed_dim = 16

        # lstm variables
        self.lstm_units = 256  # number of hidden units in the RNN
        self.dropout = .5  # keeping probability

In [3]:
from tqdm import tqdm

In [4]:
import re
import random
import pickle
from collections import Counter
import sys

import numpy as np
from gensim.models import KeyedVectors
from torch.utils.data import DataLoader, Dataset
import os

cfg = Config()

class KasreDSNew(Dataset):
    def __init__(self, cfg):
        # loading word embedding model
        try:
            handle = open(cfg.we_pickled_model_dir, 'rb') 
            self.word_embedding_model = pickle.load(handle)
        except FileNotFoundError:
            self.word_embedding_model = KeyedVectors.load_word2vec_format(cfg.we_model_dir, binary=False)
            with open(cfg.we_pickled_model_dir, 'wb') as handle:
                pickle.dump(self.word_embedding_model, handle, protocol=pickle.HIGHEST_PROTOCOL)
        self.max_len = cfg.max_len
        sents, all_ezafe_tags = self._data_reader(cfg.train_data_dir)
        print(len(sents))
        sents_shuf = []
        all_ezafe_tags_shuf = []
        index_shuf = list(range(len(sents)))

        for i in index_shuf:
            sents_shuf.append(sents[i])
            all_ezafe_tags_shuf.append(all_ezafe_tags[i])

        random.seed(17)
        random.shuffle(index_shuf)
        
        self.sents_shuf = sents_shuf
        print('data:', len(self.sents_shuf[0]))
        self.data = sents_shuf
        sents, all_ezafe_tags = self._data_reader(cfg.train_data_dir)
        self.sents = sents
        self.all_ezafe_tags = all_ezafe_tags
        
        try:
              with open('indices.pickle', 'rb')  as handle:
                self.char_to_index, self.word_to_index,  self.ezafe_tag_to_index = pickle.load(handle)
                self.index_to_word = {i: key for key, i in self.word_to_index.items()}
                self.index_to_ezafe_tag = {i: key for key, i in self.ezafe_tag_to_index.items()}
          
              

        except FileNotFoundError:
            print('Building vocabulary...')

            vocab_list = []
            char_list = []
            for sent in self.data:
                for word in sent:
                    vocab_list.append(word)
                    for char in word:
                        char_list.append(char)
            
            most_common_words = Counter(vocab_list).most_common(cfg.num_words)
            most_common_chars = Counter(char_list).most_common(cfg.num_chars)
            
            self.word_to_index = {}
            for i, pair in enumerate([('<PAD>', 0)] + most_common_words):
                self.word_to_index[pair[0]] = i + 1

            self.char_to_index = {}
            for i, pair in enumerate([('<PAD>', 0), ('<UNK>', 1)] + most_common_chars):
                self.char_to_index[pair[0]] = i + 1
            
            self.ezafe_tag_to_index = {'O': 0, 'e': 1,'ye': 2, 've':3, 'y':4, '@e':5}

            self.index_to_word = {i: key for key, i in self.word_to_index.items()}
            self.index_to_ezafe_tag = {i: key for key, i in self.ezafe_tag_to_index.items()}

            # saving the tokenizers
            with open('indices.pickle', 'wb') as handle:
                indices = self.char_to_index, self.word_to_index, self.ezafe_tag_to_index
                pickle.dump(indices, handle, protocol=pickle.HIGHEST_PROTOCOL)


    def _data_reader(self, directory):
        sents, sent = [], []
        all_ezafe_tags, ezafe_tags = [], []
        with open(directory) as bijankhan_corpus:
            for line in bijankhan_corpus:
                try:
                    if line not in ['.\tO\n', '#\tO\n']:
                        word, ezafe_tag = line.strip().split('\t')
                        sent.append(word.replace('ي', 'ی').replace('ك', 'ک').replace('ة', 'ه'))
                        ezafe_tags.append(ezafe_tag)
                    else:
                        word, ezafe_tag = line.strip().split('\t')
                        sent.append(word)
                        ezafe_tags.append(ezafe_tag)
                        if len(sent)>1:
                          if len(sent)<self.max_len :
                            all_ezafe_tags.append(ezafe_tags)
                            sents.append(sent)
                        sent = []
                        ezafe_tags = []
                except Exception as e:
                  sent = []
                  ezafe_tags = []
        print(f'data reader {len(sents)}')
        return sents, all_ezafe_tags
	

    def _pad(self, word):
        for _ in range(cfg.word_max_len - len(word)):
            word.append(0)
        return word
    

    def _sent_to_index(self, sentence, mode='word'):
        if mode is 'word':
            return [self.word_to_index.get(word, 1) for word in sentence]
        elif mode is 'char':
            indexed_sentence = []
            for word in sentence:
                indexed_word = []
                for char in word:
                    indexed_word.append(self.char_to_index.get(word, 1))
                indexed_sentence.append(self._pad(indexed_word))
            return indexed_sentence


    def _sent_to_embed(self, sentence):
        embed_sent = []
        for word in sentence:
            try:
                embed_sent.append(self.word_embedding_model[word])
            except KeyError:
                embed_sent.append([0 for _ in range(cfg.word_embed_dim)])
        return embed_sent
    
    def _ezafe_tags_to_index(self, tags):
        return [self.ezafe_tag_to_index[tag] for tag in tags]

    def __len__(self):
        return len(self.sents)

    def __getitem__(self, idx):
        char = True
        sent, ezafe_tag = self.sents[idx], self.all_ezafe_tags[idx]
        sent_char = self._sent_to_index(sent, mode='char')
        sent_word = self._sent_to_embed(sent)
        length = [1 for _ in range(len(sent))]
        tag = self._ezafe_tags_to_index(ezafe_tag)
        
        # if char:
        #     return (np.array(sent_word), np.array(sent_char), np.array(length)), np.array(tag)
        return torch.tensor(sent_word), torch.tensor(tag)


In [5]:
kasre_ds = KasreDSNew(cfg)

data reader 271410
271410
data: 23
data reader 271410


In [6]:
len(kasre_ds)

271410

In [7]:
import torch
import torch.nn as nn

In [8]:
et = [e for t in kasre_ds.all_ezafe_tags for e in t]
list(set(et))

['e', 've', 'ye', '@e', 'O', 'y']

In [8]:
x, y = kasre_ds[15118]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:168: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)


In [9]:
x.shape, y.shape

(torch.Size([100, 300]), torch.Size([100]))

In [11]:
lens = list()
for i in range(len(kasre_ds)):
  x, _ = kasre_ds[i]
  lens.append(x.shape[0])
  if x.shape[0] <1:
    print(i)

In [12]:
max(lens)

99

In [13]:
lens[20]

15

In [8]:
train_dataset, val_dataset = torch.utils.data.random_split(kasre_ds,(270410, 1000), generator=torch.Generator().manual_seed(42) )

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [10]:
train_loader = DataLoader(
    train_dataset, batch_size=1, shuffle=True, num_workers=0, pin_memory=True, 
)

test_loader = DataLoader(
    val_dataset, batch_size=1, shuffle=False, num_workers=0, pin_memory=True
)

val_loader = DataLoader(
    val_dataset, batch_size=1, shuffle=False, num_workers=0, pin_memory=True
)

In [11]:
dataloaders = {'train': train_loader,
               'test': test_loader,
               'val': val_loader}

In [12]:
class KasreAdder(nn.Module):
  def __init__(self,cfg):
    super().__init__()
    self.cfg = cfg
    self.lstm = nn.LSTM(cfg.word_embed_dim, cfg.lstm_units // 2,
                            num_layers=1, bidirectional=True, batch_first=True)
    self.hidden2tag = nn.Linear(cfg.lstm_units, cfg.num_tags)
    self.dropout = nn.Dropout()
    self.hidden = self.init_hidden(1)

  def init_hidden(self, shape):
    return (torch.randn(2, 1, self.cfg.lstm_units // 2).to(device),
            torch.randn(2, 1, self.cfg.lstm_units // 2).to(device))
    
  def forward(self, x):
    # print(f'x shape {x.shape}')
    self.hidden = self.init_hidden(x.shape[1])
    lstm_out, self.hidden = self.lstm(x, self.hidden,)
    # print(f'lstm_out {lstm_out.shape}')
    # lstm_out = lstm_out.view(len(x), self.cfg.lstm_units)
    lstm_out = self.dropout(lstm_out)
    lstm_feats = self.hidden2tag(lstm_out[0])
    return lstm_feats
    

In [13]:
model = KasreAdder(cfg).to(device)

In [14]:
lr=1e-3
epochs = 5
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=6, gamma=0.1)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, threshold=0.05, min_lr=1e-7, verbose=True, factor=0.1)
es_patience = 50

In [17]:
def train(model, optimizer, criterion, lr_scheduler, dataloaders, num_epochs=10):
  model = model.train()
  train_loss_history = list()
  val_loss_history = list()
  best_epoch = {'epoch':0, 'accuracy':0, 'loss':-100.0}
  model_path = f'checkpoints/RUN_{RUN}'
  os.mkdir(model_path)
  for epoch in range(num_epochs):
    model = model.train()
    pbar = tqdm(dataloaders['train'])
    epoch_losses = np.array([])
    for (batch, labels) in pbar:
      batch = batch.to(device, dtype=torch.float)
      labels = labels.to(device, dtype=torch.long)[0]
      optimizer.zero_grad()
      out = model(batch)
      # print(batch.shape)
      # print(labels.shape)
      # print('*************')
      # print(out.shape)
      loss = criterion(out, labels)
      loss.backward()
      optimizer.step()
      epoch_losses = np.append(epoch_losses, loss.item())
      pbar.set_description(f'epoch {epoch+1}/{num_epochs},\
                            loss= {epoch_losses.mean():.4f}')
    train_loss_history.append(epoch_losses.mean())
    # lr_scheduler.step()

    # validation
    model = model.eval()
    with torch.no_grad():
      pbar = tqdm(dataloaders['val'])
      val_loss = np.array([])
      n_samples = 0
      n_corrects = 0
      for (batch, labels) in pbar:
        batch = batch.to(device, dtype=torch.float)
        labels = labels.to(device, dtype=torch.long)[0]
        # labels = torch.tensor([l for l in labels for _ in range(30)]).to(device, dtype=torch.long)
        out = model(batch)
        loss = criterion(out, labels)
        val_loss = np.append(val_loss, loss.item())
        out = nn.Softmax()(out)
        out = out.argmax(1)
        n_corrects += (out == labels).sum()
        n_samples += out.shape[0]
      val_acc = n_corrects/n_samples
      val_loss = val_loss.mean()
      val_loss_history.append(val_loss)
      lr_scheduler.step(val_loss)
      if val_acc > best_epoch['accuracy']:
        best_epoch = {'epoch':epoch, 'accuracy':val_acc, 'loss':val_loss}
        best_model_path = os.path.join(model_path, f'best_val_checkpoint.pt')
        torch.save(model.state_dict(), best_model_path)
      print(f'epoch {epoch+1}/{num_epochs}, loss:= {val_loss.mean():.4f}, accuracy= {(n_corrects/n_samples):.4f}')
      pbar.set_description(f'epoch {epoch+1}/{num_epochs}, loss:= {val_loss.mean():.4f}, accuracy= {(n_corrects/n_samples):.4f}')
    if (sorted(val_loss_history[-es_patience:]) == val_loss_history[-es_patience:]) and (len(val_loss_history)>2*es_patience):
      break
  model.load_state_dict(torch.load(best_model_path))
  print(best_epoch)
  return model, train_loss_history, val_loss_history

In [18]:
RUN += 1
model, loss_history, val_loss_history = train(model, optimizer, criterion, lr_scheduler, dataloaders, epochs)

epoch 1/5,                            loss= 0.0958: 100%|██████████| 270410/270410 [42:01<00:00, 107.22it/s]
100%|██████████| 1000/1000 [00:03<00:00, 280.42it/s]


epoch 1/5, loss:= 0.0583, accuracy= 0.9826


epoch 2/5,                            loss= 0.0648: 100%|██████████| 270410/270410 [42:09<00:00, 106.90it/s]
100%|██████████| 1000/1000 [00:03<00:00, 281.55it/s]


epoch 2/5, loss:= 0.0521, accuracy= 0.9848


epoch 3/5,                            loss= 0.0585: 100%|██████████| 270410/270410 [43:22<00:00, 103.89it/s]
100%|██████████| 1000/1000 [00:03<00:00, 291.08it/s]


epoch 3/5, loss:= 0.0496, accuracy= 0.9852


epoch 4/5,                            loss= 0.0555: 100%|██████████| 270410/270410 [45:01<00:00, 100.08it/s]
100%|██████████| 1000/1000 [00:03<00:00, 293.40it/s]


epoch 4/5, loss:= 0.0480, accuracy= 0.9862


epoch 5/5,                            loss= 0.0534: 100%|██████████| 270410/270410 [45:10<00:00, 99.77it/s] 
100%|██████████| 1000/1000 [00:03<00:00, 285.20it/s]


epoch 5/5, loss:= 0.0450, accuracy= 0.9869
{'epoch': 4, 'accuracy': tensor(0.9869, device='cuda:0'), 'loss': 0.044961439092578444}


In [19]:
!cp checkpoints/RUN_22/best_val_checkpoint.pt drive/MyDrive/kasre_model.pt

In [20]:
sent = 'توپ سبز رنگ'.split()
sent_words = kasre_ds._sent_to_embed(sent)
sent_words = torch.tensor(sent_words).to(device)
sent_words.shape

torch.Size([3, 300])

In [21]:
with torch.no_grad():
  out = model(sent_words.unsqueeze(0))


In [22]:
out.shape

torch.Size([3, 6])

In [23]:
out = nn.Softmax()(out)
out = out.argmax(1)
out

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


tensor([1, 0, 1], device='cuda:0')

In [ ]:
for word in sent:
  print(word)

حال
شما
چطور
است
